# Analyse Comparative des Tokenizers sur des Données Textuelles Multilingues


## Import des librairies nécessaires au chargement et analyse des données

- os: donne accès utiles pour le chargement des données du dataset
- json: chargement des phrases tokenisées stockées au format json
- pandas: chargement des statistiques sur le dataset ainsi que statistiques issues de wikipedia
- numpy: manipulation de données vectorielles
- scipy: librairie de fonction scientifiques
- iso639: permet la manipulation des codes iso-639 identifiant des langues
- transformers: permet l'utilisation des tokenizers mise à disposition sur [Hugging Face](https://huggingface.co/)
- matplotlib: permet la visualisation de données


In [ ]:
# Install from the requirements file
!pip install -r requirements.txt

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import scipy
import iso639
from transformers import AutoTokenizer
import matplotlib.pyplot as plt

## Définition des chemins vers les données à charger

- LANG_INFO_PATH: chemin vers le tableau contenant les informations sur les langues dont les phrases ont étés pré-tokenisées
- FLORES_DATASET_PATH: chemin vers le dossier content le jeu de données FLORES-200
- MADLAD_STATS_PATH: chemin vers le tableau contenant les statistiques sur les données utilisées à l'entraînement du modèle MADLAD-400
- WIKI_STATS_PATH: chemin vers le tableau content les statistiques sur wikipedia


In [ ]:
LANG_INFO_PATH = './inputs/FLORES-200.lang_info.csv'
FLORES_DATASET_PATH = './inputs/flores200_dataset/'
MADLAD_STATS_PATH = './inputs/madlad_stats.tsv'
WIKI_STATS_PATH = './inputs/wikipedia_stats.csv'

## Selection de la partition de FLORES-200

Le jeu de données FLORES-200 est séparé en deux partition:

- dev
- devtest

La variable `SPLIT` permettent de selectionner la partition à charger.


In [ ]:
SPLIT = 'dev'

## Selection des données pré-tokenisées à charger

Le jeu de données FLORES-200 a été tokenisé à l'aide des tokenizer des modèles suivant:

- [NLLB](https://huggingface.co/docs/transformers/model_doc/nllb)
- [MADLAD-400](https://huggingface.co/docs/transformers/model_doc/madlad-400)

La selection du modèle considéré se fait à travers la variable `MODEL`


In [ ]:
MODEL = 'madlad' # 'nllb'

Chemin vers le fichier json contenant les données pré-tokenisées:


In [ ]:
DATASET_TOKENIZED_PATH = './inputs/FLORES-200.{:}.tokenized.json'.format(MODEL)

## Mise en correspondance du modèle sélectionné avec son identifiant unique

Le code suivant met en correspondance le nom court du modèle utilisé (`MODEL`) avec son identifiant (`MODEL_NAME`) sur Hugging Face.
Cet identifiant sera utilisé plus tard pour charger le tokenizer du modèle sélectionné, ce qui permettra de:

- décoder les phrases pré-tokenisées
- encoder des phrases pour des nouvelles phrases
- avoir accès aux identifiants de token spéciaux, utiles lors de l'analyse des phrases tokenisées


In [ ]:
if MODEL == 'madlad':
    MODEL_NAME = 'google/madlad400-3b-mt'
    
elif MODEL == 'nllb':
    MODEL_NAME = 'facebook/nllb-200-3.3B'

## Chargement des informations sur les données pré-calculées

Le tableau csv pointé par `LANG_INFO_PATH` liste les langues pour lesquelles les phrases ont déjà été tokenisées associées avec différentes informations utiles, notamment:

- lang_script: le code iso-639-3 ainsi que le script utilisé, cette combinaison correspondant au nom de fichier dans le jeu de données FLORES-200
- lang_family: la famille à laquelle apartient la langue


In [ ]:
lang_info_list = pd.read_csv(LANG_INFO_PATH, sep='\t', index_col=0)

In [ ]:
lang_info_list

In [ ]:
lang_info_list = pd.read_csv(LANG_INFO_PATH, sep='\t', index_col=0)

## Chargement des phrases du dataset


In [ ]:
# Initialisation du dictionnaire contenant les phrases du dataset pour chaque langue
dataset = dict()

# Construction du chemin vers le jeu de données à partir de la partition choisie
split_base_path = os.path.join(FLORES_DATASET_PATH, SPLIT)


# Pour chaque langue identifié par l'index du tableau lang_info_list
for lang_id, lang_info in lang_info_list.iterrows():
    
    # Construction du chemin vers le fichier listant les phrases pour la langue "lang_id"
    file_path = os.path.join(split_base_path,  '{:}.{:}'.format(lang_info['lang_script'], SPLIT))

    # Chargement du fichier listant les phrases
    with open(file_path, 'r') as file:
        dataset[lang_id] = file.read().split('\n')

        # Suppression de la dernière ligne vide
        del(dataset[lang_id][-1])


In [ ]:
dataset

## Chargement des statistiques sur les wikipedias en différentes langues


In [ ]:
wiki_stat_list = pd.read_csv(WIKI_STATS_PATH, sep='\t', index_col=0)

### Nettoyage des statistiques wikipedia

Les lignes ne correspondant pas à un wikipedia d'une langue données sont supprimées et le code iso-639-3 est ajouté


In [ ]:
# Ajout de la colonne lang_id correspondant au code iso-639-3 pour les langues du tableau
wiki_stat_list['lang_id'] = ''

# Parcours des identifiants des lignes du tableau
for site_id in wiki_stat_list.index:
    # Suppression de la ligne si elle ne correspond pas à un sous-domaine de wikipedia
    if not site_id.endswith('.wikipedia'):
        wiki_stat_list.drop(site_id, inplace=True)
        continue

    # Extraction du préfix du sous-domaine de wikipedia
    site_prefix = site_id.split('.')[0]
    
    lang_id = None

    # La capture d'exception est nécessaire ici dans le cas ou le prefix n'est pas décodable comme code iso-639
    try:
        # Identification du type de code iso correspondant à la langue
        if len(site_prefix) == 2:
            # iso-639 part 1
            lang_id = iso639.Language.from_part1(site_prefix).part3
            
        elif len(site_prefix) == 3:
            # iso-639 part 3
            lang_id = iso639.Language.from_part3(site_prefix).part3

        else:
            # Trop long pour être un code iso-639
            wiki_stat_list.drop(site_id, inplace=True)
            continue
            
    except Exception as e:
        # Le préfix n'a pas pu être décodé comme code iso-639
        wiki_stat_list.drop(site_id, inplace=True)
        continue

    # Ajout de l'identifiant iso-639 pour le sous-domaine
    wiki_stat_list.loc[site_id, 'lang_id'] = lang_id

### Utilisation du code iso-639 comme index du tableau


In [ ]:
wiki_stat_list = wiki_stat_list.reset_index().set_index('lang_id')

In [ ]:
wiki_stat_list

## Chargement des statistiques sur les données utilisées à l'entraînement du modèle MADLAD-400


In [ ]:
madlad_stat_list = pd.read_csv(MADLAD_STATS_PATH, sep='\t', index_col=0)

In [ ]:
madlad_stat_list

## Chargement des données pré-tokenisées

### Structure des données

Les données tokenisées son représentées dans un dictionnaire au format json structuré de la manière suivante:

- dictionnaire des phrases pour chaque langue: `dict<lang_id, list>`
  - liste des phrases tokenisées pour une langue donnée: `list<dict>`
    - données brutes issues du tokenizer pour une phrase donnée: `dict<string, list>`
      - liste des tokens identifiée par la clé `"input_ids"`: `list<list<int>>`
      - masque d'attention identifié par la clé `"attention_mask"`: `list<list<int>>`

### Accès à une phrase tokenisée

L'accès aux tokens générés pour la première phrase en Anglais se fera par exemple de la manière suivante:

```
dataset_tokenized['eng'][0]['input_ids'][0]
```

### Attention

- Les données issues du tokenizer ont été stockées telle quelle en suivant un format de batch, elles sont donc représentées par un tableau imbriqué.
- Des tokens spéciaux ont automatiquement été insérés, il faudra donc les retirer lors de calcul de charactéristiques des phrases tokenisées. Ces tokens sont par exemple:
  - `tokenizer.eos_token_id`: identifiant de début de phrase
  - `tokenizer.eos_token_id`: identifiant de fin de phrase
  - `tokenizer.unk_token_id`: identifiant de token inconnu


In [ ]:
with open(DATASET_TOKENIZED_PATH, 'r') as fd:
    dataset_tokenized = json.load(fd)[SPLIT]

In [ ]:
dataset_tokenized['eng'][0]['input_ids'][0]

## Calcul d'une charactéristique pour une phrase donnée

Cette section montre un exemple de calcul de charactéristique à partir d'une phrase sous forme de chaine de charactères (string) et de sa version tokenisée (list\<int\>).

La charactéristique calculée ici est le nombre de token moyen produit par mot, son calcul est défini dans une fonction afin de pouvoir être réutilisé plus tard.


In [ ]:
def compute_token_per_word_ratio(sentence, sentence_token_list):
    # Comptage du nombre de mots
    sentence_word_count = len(sentence.split())
    
    # Comptage du nombre de tokens
    sentence_token_count = len(sentence_token_list)
    
    # Calcul du nombre de tokens produits par mots
    sentence_token_per_word_ratio = sentence_token_count / sentence_word_count

    return sentence_token_per_word_ratio


In [ ]:
# Choix de la langue pour la phrase
lang_id = 'eng'

# Choix du numéro de phrase
sentence_index = 0

# Récuperation de la phrase selectionnée
sentence = dataset[lang_id][sentence_index]

# Récupération de la liste des tokens générée pour la phrase
sentence_token_list = dataset_tokenized[lang_id][sentence_index]['input_ids'][0]

# Calcul de la charactéristique
sentence_token_per_word_ratio = compute_token_per_word_ratio(sentence, sentence_token_list)

# Affichage de la charactéristique
sentence_token_per_word_ratio

## Utilisation du tokenizer

Cette section montre comment un tokenizer peut être chargé depuis huggingface et utilisé pour décoder et encoder des phrase.

La [documentation des tokenizer huggingface](https://huggingface.co/docs/transformers/main_classes/tokenizer) peut être utile ici.


### Chargement du tokenizer pour le modèle considéré

la variable MODEL_NAME correspond ici à l'identifiant Hugging Face du modèle pour lequel le tokenizer est chargé, la variable MODEL_NAME peut être remplacée par n'importe quel identifiant de modèle disponible sur Hugging Face.

Par exemple pour charger le tokenizer du modèle Phi-3 de Microsoft:

```
tokenizer = AutoTokenizer.from_pretrained(microsoft/Phi-3-mini-128k-instruct)
```


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Exemple de décodage d'une phrase


In [ ]:
# Choix de la langue pour la phrase
lang_id = 'eng'

# Choix du numéro de phrase
sentence_index = 0

# Récuperation de la liste des tokens pour la phrase selectionnée
sentence_token_list = dataset_tokenized[lang_id][sentence_index]['input_ids'][0]

# Décodage à l'aide du tokenizer
sentence_decoded = tokenizer.decode(sentence_token_list, skip_special_tokens=True)

# Affichage de la phrase décodée
sentence_decoded

### Exemple d'encodage d'une phrase


In [ ]:
# Choix de la langue pour la phrase
lang_id = 'eng'

# Choix du numéro de phrase
sentence_index = 0

# Récuperation de la phrase selectionnée
sentence = dataset[lang_id][sentence_index]

# Décodage à l'aide du tokenizer
# L'argument add_special_tokens=False permet d'éviter de générer les tokens spéciaux tel que les tokens de début et fin de phrase
sentence_encoded = tokenizer.encode(sentence, add_special_tokens=False)

# Affichage de la phrase décodée
sentence_encoded

### Décodage token par token d'une phrase


In [ ]:
# Choix de la langue pour la phrase
lang_id = 'eng'

# Choix du numéro de phrase
sentence_index = 0

for token_id in dataset_tokenized[lang_id][sentence_index]['input_ids'][0]:
    decoded_token = tokenizer.decode([token_id], skip_special_tokens=True)
    print('{:}:\t"{:}"'.format(token_id, decoded_token))

### Encodage mot par mot d'une phrase


In [ ]:
# Choix de la langue pour la phrase
lang_id = 'eng'

# Choix du numéro de phrase
sentence_index = 0

# Récupération de la phrase
sentence = dataset[lang_id][sentence_index]

# Séparation de la phrase en mots
sentence_word_list = sentence.split()

for word in sentence_word_list:
    word_token_list = tokenizer.encode(word, add_special_tokens=False)
    print('"{:}":\n\t{:}'.format(word, word_token_list))

### Encodage du jeu de données entier à l'aide du tokenizer nouvellement chargé

Cette section montre comment l'ensemble du jeu de données peut être encodé à l'aide du tokenizer nouvellement chargé.

#### Attention

Les données produites ici ne seront pas identiques à celles présentes dans `dataset_tokenized` pour les raisons suivantes:

- L'encodage se fait ici phrase par phrase et non par batch, produisant un liste simple de token (list\<int\>) et non une liste imbriquée (list\<list\<int\>\>)
- l'encodage phrase par phrase ne génére qu'une liste de token et non un dictionnaire contenant la liste de token (`input_ids`) et le masque (`attention_mask`).
- La génération de token spéciaux tel que `tokenizer.eos_token_id` et `tokenizer.bos_token_id` est désactivée

La structure produit aura le format suivant:

- dictionnaire des phrases pour chaque langue: `dict<lang_id, list>`
  - liste des phrases tokenisées pour une langue donnée: `list<list>`
    - liste des tokens: `list<int>`

#### Accès à une phrase tokenisée

L'accès aux tokens générés pour la première phrase en Anglais se fera par exemple de la manière suivante:

```
dataset_tokenized_new['eng'][0]
```

La ou pour les données pré-calculées l'accès se faisait de la manière suivante:

```
dataset_tokenized_new['eng'][0]['input_ids'][0]
```


In [ ]:
dataset_tokenized_new = dict()

# Pour chaque langue identifié par l'index du tableau lang_info_list
for lang_id in lang_info_list.index:
    # Insertion de la liste contenant les phrases encodées pour la langue
    dataset_tokenized_new[lang_id] = list()

    # Parcours des phrases pour la langue donnée
    for sentence in dataset[lang_id]:
        # Encodage de la phrase
        sentence_token_list = tokenizer.encode(sentence, add_special_tokens=False)

        # Enregistrement de la liste des tokens
        dataset_tokenized_new[lang_id].append(sentence_token_list)

### Comparaison avec les données pré-calculés


In [ ]:
print('Données précalculées:')
print(dataset_tokenized['eng'][0]['input_ids'][0])

In [ ]:
print('Données fraîchement calculées:')
print(dataset_tokenized_new['eng'][0])

## Mise en pratique

Dans cette section vous devrez utiliser les connaissances acquises dans les sections précédentes et:

- calculer des charactéristiques des phrases du jeu de données
- analyser ces charactéristiques
- visualiser ces charactéristiques


### Calcul de charactéristiques

Remplir le dictionnaire `dataset_features` avec les valeurs d'une charactéristique dont vous aurez défini le calcul


In [ ]:
def compute_feature_1(sentence, token_list):
    return len(sentence) * (1 + np.random.rand())


def compute_feature_2(sentence, token_list):
    return len(sentence) * (-1 + np.random.rand())


dataset_feature_list = dict()

dataset_feature_list['feature_1'] = dict()
dataset_feature_list['feature_2'] = dict()

for lang_id in lang_info_list.index:
    dataset_feature_list['feature_1'][lang_id] = list()
    dataset_feature_list['feature_2'][lang_id] = list()

    for sentence, token_list in zip(dataset[lang_id], dataset_tokenized_new[lang_id]):
        sentence_feature_1 = compute_feature_1(sentence, token_list)
        sentence_feature_2 = compute_feature_2(sentence, token_list)
        
        dataset_feature_list['feature_1'][lang_id].append(sentence_feature_1)
        dataset_feature_list['feature_2'][lang_id].append(sentence_feature_2)
        

### Analyse de charactéristiques


In [ ]:
# Mise à plat des données
dataset_feature_list_flat = dict()


for feature in dataset_feature_list:
    dataset_feature_list_flat[feature] = list()
    for lang_id in sorted(lang_info_list.index):
        dataset_feature_list_flat[feature].extend(dataset_feature_list[feature][lang_id])

feature_1_2_pearson = scipy.stats.pearsonr(dataset_feature_list_flat['feature_1'], dataset_feature_list_flat['feature_2'])

feature_1_2_pearson

### Visualisation des charactéristiques


In [ ]:
plt.figure()
plt.scatter(dataset_feature_list_flat['feature_1'], dataset_feature_list_flat['feature_2'], s=1)

plt.show()